In [33]:
import sys
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')
#warnings.filterwarnings(action='once')
#!pip install tqdm #--user
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
#!pip install gensim #--user
from gensim.models import Doc2Vec
from sklearn import utils
from sklearn.model_selection import train_test_split
import gensim
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import TaggedDocument
import re
import seaborn as sns
import matplotlib.pyplot as plt

First attempt in doc2vec

In [34]:
supp_df=pd.read_csv("meetup.csv")

cat_list=list(supp_df.category.unique())
cat_list=[str(elem) for elem in cat_list]
cat_list.sort()
cat_list=[elem.replace("/", " ") for elem in cat_list]
"""
cat_list_old= cat_list[:]
cat_list=[]
for elem in cat_list_old:
    if " " in elem:
        els=elem.split(" ")
        cat_list.append(els)
    else:
        cat_list.append(elem)
"""
cat_list

df=pd.read_csv("processed_description_sine_error.csv")

df.head()

df["category"]="NaN"

df["category"]= df.apply(lambda r: cat_list[r.category_bin], axis=1)

df

,Unnamed: 0,description,category_bin,lang_ok,processed_description,category
0,0,<p>FOOD AND FELLOWSHIP | 5 PM</p> \n<p>CONCERT...,27,english,food fellowship pm concert pm may th join us m...,sci-fi fantasy
1,1,<p><b>【WhyNot!?JAPAN + MeetUp Collaboration Ev...,28,english,whynot japan meetup collabor event peopl come ...,singles
2,2,<p>TENTATIVE<br></p>\n<p><br></p>\n<p>10.30 - ...,31,english,tentat introduct orient session break network ...,support
3,3,"<p><span>Hey explorersssss, what's up?</span><...",15,english,hey explorersssss say night view hong kong vic...,language ethnic identity
4,4,"<p>Free, unauditioned, collaborative pop choir...",18,english,free unaudit collabor pop choir identifi women...,music
5,6,"<p>We provide a forum to learn about, promote ...",3,english,provid forum learn promot grow member busi,career business
6,7,<p><b>【WhyNot!?JAPAN + MeetUp Collaboration Ev...,28,english,whynot japan meetup collabor event peopl come ...,singles
7,8,<p>This is an introductory meeting to get to k...,31,english,introductori meet get know fellow coder share ...,support
8,9,"<p>The Future of the IoT with Rajeev Suri, CEO...",31,english,futur iot rajeev suri ceo kathrin buvac chief ...,support
9,10,"<p><img src=""https://secure.meetupstatic.com/p...",13,english,breakthrough weight loss wk workshop lose weig...,health wellbeing


In [ ]:
process_all=False
#process_all=True

if process_all:
    train, test = train_test_split(df, test_size=0.3, random_state=42)
else:    
    train, test = train_test_split(df[:10000], test_size=0.3, random_state=42)

In [36]:
train_tagged = train.apply(
    lambda r: TaggedDocument(words=str(r.processed_description).split(" "), tags=[r.category]), axis=1)
test_tagged = test.apply(
    lambda r: TaggedDocument(words=str(r.processed_description).split(" "), tags=[r.category]), axis=1)

In [37]:
#train_tagged.values[30]

In [38]:
import multiprocessing

cores = multiprocessing.cpu_count()
cores
cores=4

model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, hs=0, min_count=2, sample = 0, workers=cores)

model_dbow.build_vocab([x for x in tqdm(train_tagged.values)])

100%|██████████| 114455/114455 [00:00<00:00, 3673040.63it/s]


In [39]:
%%time
for epoch in range(10):
    model_dbow.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

100%|██████████| 114455/114455 [00:00<00:00, 3906635.29it/s]


CPU times: user 5min 46s, sys: 4.42 s, total: 5min 51s
Wall time: 1min 38s


In [40]:
def vec_for_learning(model, tagged_docs):
    #sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in tqdm(tagged_docs.values)])
    return targets, regressors

In [41]:
y_train, X_train = vec_for_learning(model_dbow, train_tagged)
y_test, X_test = vec_for_learning(model_dbow, test_tagged)

100%|██████████| 49053/49053 [03:01<00:00, 270.93it/s]


In [53]:
go=False
if go:
    import joblib
    if process_all:
        joblib.dump(model_dbow, "model_dbow_all.joblib")
        joblib.dump(X_train, "X_train_all.joblib")
        joblib.dump(y_train, "y_train_all.joblib")
        joblib.dump(X_test, "X_test_all.joblib")
        joblib.dump(y_test, "y_test_all.joblib")

In [54]:
logreg = LogisticRegression(n_jobs=5, C=1e5)

In [ ]:
logreg.fit(X_train, y_train)

In [20]:
y_pred = logreg.predict(X_test)

In [21]:
from sklearn.metrics import accuracy_score, f1_score

print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))

Testing accuracy 0.49433333333333335
Testing F1 score: 0.4875738319385951


In [44]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import classification_report, roc_curve, auc

In [45]:
forest = RandomForestClassifier(n_jobs=4) 
forest = forest.fit(X_train, y_train)

In [46]:
#X_test_vec_tf = tf_vectorizer.transform(X_test)
#X_test_vec_tf = X_test_vec_tf.toarray()

In [49]:
forest.score(X_test, y_test)#100feat

0.6645668970297434

In [51]:
predictions = forest.predict(X_test)


In [52]:
print(metrics.classification_report(y_test, predictions)) #100 features

                          precision    recall  f1-score   support

                    LGBT       0.48      0.39      0.43       457
   alternative lifestyle       0.86      0.70      0.78       223
              book clubs       0.41      0.46      0.43       332
         career business       0.64      0.80      0.71      4670
        cars motorcycles       0.67      0.46      0.54       373
   community environment       0.53      0.42      0.47       864
                 dancing       0.71      0.76      0.74      1073
      education learning       0.55      0.43      0.48       969
          fashion beauty       0.52      0.23      0.32       102
       fine arts culture       0.61      0.57      0.59      1051
                 fitness       0.62      0.67      0.65      2288
              food drink       0.55      0.51      0.53      1503
                   games       0.72      0.77      0.75      1455
        health wellbeing       0.64      0.74      0.69      4574
         